In [1]:
# Standard libraries for system operations and numerical processing
import sys
import numpy as np

# GDAL: Library for reading and processing geospatial data formats
from osgeo import gdal
gdal.UseExceptions()

# Matplotlib: Optional library for plotting (if needed for additional visualization)
import matplotlib.pyplot as plt

# PyQt5 modules for building the graphical user interface (GUI)
from PyQt5.QtWidgets import (
    QApplication, QLabel, QMainWindow, QVBoxLayout, QWidget,
    QLineEdit, QPushButton, QHBoxLayout, QScrollArea
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QColor, QPen  # GUI elements and drawing tools
from PyQt5.QtCore import Qt, QEvent  # Core Qt functionalities and event handling


In [ ]:
# Define the file path for the GeoTIFF image
file_path = "testGeoTiff.tif"

# Open the GeoTIFF file using GDAL
dataset = gdal.Open(file_path)
if dataset is None:
    raise FileNotFoundError(f"Unable to open {file_path}")

# Print basic metadata: image dimensions, number of bands, and pixel resolution

# Get image dimensions (width and height in pixels)
width = dataset.RasterXSize
height = dataset.RasterYSize
print(f"Image Dimensions: {width} x {height}")

# Get the number of bands in the dataset
num_bands = dataset.RasterCount
print(f"Number of Bands: {num_bands}")

# Retrieve geotransform parameters to extract pixel resolution information.
# The geotransform returns a tuple: 
# (top left x, pixel width, rotation, top left y, rotation, pixel height)
geotransform = dataset.GetGeoTransform()
if geotransform:
    pixel_width = geotransform[1]
    # The pixel height is usually negative; take the absolute value for resolution.
    pixel_height = abs(geotransform[5])
    print(f"Pixel Resolution: {pixel_width} (width) x {pixel_height} (height)")
else:
    print("No geotransform information available.")

# Read the first three bands (assuming the image is an RGB composite)
band1 = dataset.GetRasterBand(1).ReadAsArray()
band2 = dataset.GetRasterBand(2).ReadAsArray()
band3 = dataset.GetRasterBand(3).ReadAsArray()

# Stack the three bands along the third axis to form an RGB image array
rgb_image = np.dstack((band1, band2, band3))

# Display the RGB image in a separate window using matplotlib
plt.figure()
plt.imshow(rgb_image)
plt.title("Landsat Image")
plt.show()


In [3]:
# Cell 2: Define a QLabel subclass that supports zooming and panning,
# and that updates live coordinate displays in the main viewer.
class ZoomImageLabel(QLabel):
    """
    A QLabel subclass that supports image zooming (with the mouse wheel)
    and panning (by dragging). It communicates with the main viewer to update
    coordinate displays.
    """
    def __init__(self, parentViewer, parent=None):
        super().__init__(parent)
        self.parentViewer = parentViewer  # Reference to the main viewer.
        self.setMouseTracking(True)
        self.setFocusPolicy(Qt.StrongFocus)  # Ensures wheel events are captured.
        self.zoomFactor = 1.0
        self.isDragging = False
        self.lastMousePos = None
        self.scrollAreaRef = None  # To be set by the main viewer.
        self.originalPixmap = None  # Store the full-resolution image.

    def setPixmap(self, pixmap):
        """
        Overridden to store the original pixmap for zooming operations.
        """
        super().setPixmap(pixmap)
        if self.originalPixmap is None:
            self.originalPixmap = pixmap

    def wheelEvent(self, event):
        """
        Zooms in or out based on the mouse wheel movement.
        """
        delta = event.angleDelta().y()
        zoomStep = 1.25 if delta > 0 else 0.8
        self.zoomFactor *= zoomStep

        if self.originalPixmap:
            newSize = self.originalPixmap.size() * self.zoomFactor
            scaledPixmap = self.originalPixmap.scaled(newSize, Qt.KeepAspectRatio, Qt.SmoothTransformation)
            super().setPixmap(scaledPixmap)
            self.resize(scaledPixmap.size())

        event.accept()

    def eventFilter(self, source, event):
        """
        Forwards wheel events from the scroll area's viewport to this widget.
        """
        if event.type() == QEvent.Wheel:
            self.wheelEvent(event)
            return True
        return super().eventFilter(source, event)

    def mousePressEvent(self, event):
        """
        Starts panning with left-click; on right-click, transfers the current
        coordinates to the marker input fields.
        """
        if event.button() == Qt.LeftButton:
            self.isDragging = True
            self.lastMousePos = event.globalPos()
            event.accept()
        elif event.button() == Qt.RightButton:
            self.parentViewer.latInput.setText(self.parentViewer.latDisplay.text())
            self.parentViewer.lonInput.setText(self.parentViewer.lonDisplay.text())
            event.accept()
        else:
            super().mousePressEvent(event)

    def mouseMoveEvent(self, event):
        """
        If dragging, pans the image; otherwise, updates the coordinate displays.
        """
        if self.isDragging and self.scrollAreaRef:
            delta = event.globalPos() - self.lastMousePos
            self.lastMousePos = event.globalPos()
            hBar = self.scrollAreaRef.horizontalScrollBar()
            vBar = self.scrollAreaRef.verticalScrollBar()
            hBar.setValue(hBar.value() - delta.x())
            vBar.setValue(vBar.value() - delta.y())
            event.accept()
        else:
            pos = event.pos()
            if self.pixmap() and 0 <= pos.x() < self.pixmap().width() and 0 <= pos.y() < self.pixmap().height():
                originalX = pos.x() / self.zoomFactor
                originalY = pos.y() / self.zoomFactor
                # Convert the image pixel coordinates to geographic coordinates.
                lon, lat = gdal.ApplyGeoTransform(self.parentViewer.geoTransform, originalX, originalY)
                self.parentViewer.latDisplay.setText(f"{lat:.4f}")
                self.parentViewer.lonDisplay.setText(f"{lon:.4f}")
                self.parentViewer.xDisplay.setText(str(int(originalX)))
                self.parentViewer.yDisplay.setText(str(int(originalY)))
            else:
                self.parentViewer.latDisplay.clear()
                self.parentViewer.lonDisplay.clear()
                self.parentViewer.xDisplay.clear()
                self.parentViewer.yDisplay.clear()
            super().mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        """
        Ends the panning operation on left mouse button release.
        """
        if event.button() == Qt.LeftButton:
            self.isDragging = False
            event.accept()
        else:
            super().mouseReleaseEvent(event)


In [4]:
# Cell 3: Define the main viewer class for the GeoTIFF application.
class TiffViewer(QMainWindow):
    """
    Main window class for displaying a GeoTIFF image with live coordinate tracking,
    location marking, and zoom/pan functionality.
    """
    def __init__(self, tiffFilePath):
        super().__init__()
        self.tiffDataset = gdal.Open(tiffFilePath)
        if self.tiffDataset is None:
            raise FileNotFoundError(f"Unable to open {tiffFilePath}")

        self.geoTransform = self.tiffDataset.GetGeoTransform()
        self.inverseTransform = gdal.InvGeoTransform(self.geoTransform)  # Reverse geotransform.
        self.imagePixmap = self.loadImage()
        self.basePixmap = self.imagePixmap.copy()  # Preserve the original image.
        self.markers = []  # List to store marked pixel positions.
        self.setupUI()

    def setupUI(self):
        """
        Constructs the user interface, including the image display and a right-hand panel
        split into a top description section and bottom controls.
        """
        self.setWindowTitle("GeoTIFF Coordinate Viewer")
        imgWidth = self.imagePixmap.width()
        imgHeight = self.imagePixmap.height()

        # Set the window geometry with extra space on the right.
        self.setGeometry(100, 100, imgWidth + 300, imgHeight)
        mainLayout = QHBoxLayout()

        # Left panel: Contains the image and live coordinate displays.
        leftPanel = QVBoxLayout()
        coordLayout = QHBoxLayout()
        self.latDisplay = QLineEdit(); self.lonDisplay = QLineEdit()
        self.xDisplay = QLineEdit(); self.yDisplay = QLineEdit()

        # Make the coordinate fields read-only.
        for field in [self.latDisplay, self.lonDisplay, self.xDisplay, self.yDisplay]:
            field.setReadOnly(True)

        coordLayout.addWidget(QLabel("Lat:"))
        coordLayout.addWidget(self.latDisplay)
        coordLayout.addWidget(QLabel("Lon:"))
        coordLayout.addWidget(self.lonDisplay)
        coordLayout.addWidget(QLabel("X:"))
        coordLayout.addWidget(self.xDisplay)
        coordLayout.addWidget(QLabel("Y:"))
        coordLayout.addWidget(self.yDisplay)
        leftPanel.addLayout(coordLayout)

        # Create the zoomable image label.
        self.imageDisplay = ZoomImageLabel(parentViewer=self)
        self.imageDisplay.setPixmap(self.imagePixmap)
        self.imageDisplay.resize(self.imagePixmap.size())
        self.imageDisplay.setScaledContents(True)

        # Add the image label to a scroll area to enable panning.
        scrollArea = QScrollArea()
        scrollArea.setWidget(self.imageDisplay)
        scrollArea.setWidgetResizable(False)
        scrollArea.viewport().installEventFilter(self.imageDisplay)
        self.imageDisplay.scrollAreaRef = scrollArea
        leftPanel.addWidget(scrollArea)

        # Right panel: Divided into a top description and bottom control section.
        rightPanelMain = QVBoxLayout()

        # Top section: Application description with enhanced styling.
        descriptionLabel = QLabel(
            "<h2>GeoTIFF Visualizer</h2>"
            "<p style='font-size:14px;'>"
            "Explore and analyze geospatial imagery with powerful features:<br>"
            "- Zoom &amp; Pan<br>"
            "- Live Coordinate Tracking<br>"
            "- Location Marking"
            "</p>"
        )
        descriptionLabel.setAlignment(Qt.AlignCenter)
        descriptionLabel.setWordWrap(True)
        descriptionLabel.setStyleSheet(
            "background-color: #f7f7f7; "
            "color: #333333; "
            "border: 2px solid #cccccc; "
            "border-radius: 10px; "
            "padding: 15px;"
        )
        rightPanelMain.addWidget(descriptionLabel)

        # Add a stretch to push the bottom section down.
        rightPanelMain.addStretch(1)

        # Bottom section: Marker input fields and buttons.
        bottomControls = QVBoxLayout()
        bottomControls.setAlignment(Qt.AlignBottom)
        bottomControls.addWidget(QLabel("Enter Latitude:"))
        self.latInput = QLineEdit()
        bottomControls.addWidget(self.latInput)
        bottomControls.addWidget(QLabel("Enter Longitude:"))
        self.lonInput = QLineEdit()
        bottomControls.addWidget(self.lonInput)
        self.markButton = QPushButton("Mark Location")
        bottomControls.addWidget(self.markButton)
        self.markButton.clicked.connect(self.markLocation)
        self.resetZoomButton = QPushButton("Reset Zoom")
        bottomControls.addWidget(self.resetZoomButton)
        self.resetZoomButton.clicked.connect(self.resetZoom)
        bottomControls.setSpacing(10)

        bottomContainer = QWidget()
        bottomContainer.setLayout(bottomControls)
        rightPanelMain.addWidget(bottomContainer)

        # Wrap the right panel in a container with fixed width.
        rightPanelContainer = QWidget()
        rightPanelContainer.setLayout(rightPanelMain)
        rightPanelContainer.setFixedWidth(300)

        mainLayout.addLayout(leftPanel)
        mainLayout.addWidget(rightPanelContainer)
        centralContainer = QWidget()
        centralContainer.setLayout(mainLayout)
        self.setCentralWidget(centralContainer)

    def loadImage(self):
        """
        Loads the GeoTIFF image, normalizes its bands, and returns a QPixmap.
        """
        width = self.tiffDataset.RasterXSize
        height = self.tiffDataset.RasterYSize
        redBand = self.tiffDataset.GetRasterBand(1).ReadAsArray()
        greenBand = self.tiffDataset.GetRasterBand(2).ReadAsArray()
        blueBand = self.tiffDataset.GetRasterBand(3).ReadAsArray()

        def normalize(arr):
            arrMin, arrMax = np.min(arr), np.max(arr)
            if arrMax > arrMin:
                return ((arr - arrMin) / (arrMax - arrMin) * 255).astype(np.uint8)
            else:
                return np.zeros_like(arr, dtype=np.uint8)

        redBand = normalize(redBand)
        greenBand = normalize(greenBand)
        blueBand = normalize(blueBand)
        rgbArray = np.dstack((redBand, greenBand, blueBand))
        rgbArray = np.ascontiguousarray(rgbArray)
        qImg = QImage(rgbArray.data, width, height, 3 * width, QImage.Format_RGB888)
        return QPixmap.fromImage(qImg)

    def markLocation(self):
        """
        Marks a location on the image based on user-entered geographic coordinates.
        Converts these coordinates to pixel positions and updates the display.
        """
        try:
            latValue = float(self.latInput.text())
            lonValue = float(self.lonInput.text())
            pixelX, pixelY = gdal.ApplyGeoTransform(self.inverseTransform, lonValue, latValue)
            self.markers.append((int(pixelX), int(pixelY)))
            self.refreshImage()
        except ValueError:
            print("Invalid coordinate input. Please enter numeric values.")

    def refreshImage(self):
        """
        Redraws the image with any marked locations.
        """
        updatedPixmap = self.basePixmap.copy()
        painter = QPainter(updatedPixmap)
        markerPen = QPen(QColor(255, 0, 0), 3)
        painter.setPen(markerPen)
        for x, y in self.markers:
            painter.drawLine(x - 5, y - 5, x + 5, y + 5)
            painter.drawLine(x - 5, y + 5, x + 5, y - 5)
        painter.end()

        # Update the zoomable image label with the new pixmap.
        self.imageDisplay.originalPixmap = updatedPixmap
        newSize = updatedPixmap.size() * self.imageDisplay.zoomFactor
        scaledPixmap = updatedPixmap.scaled(newSize, Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.imageDisplay.setPixmap(scaledPixmap)
        self.imageDisplay.resize(scaledPixmap.size())

    def resetZoom(self):
        """
        Resets the zoom factor to 1.0 and updates the image display.
        """
        self.imageDisplay.zoomFactor = 1.0
        if self.imageDisplay.originalPixmap:
            self.imageDisplay.setPixmap(self.imageDisplay.originalPixmap)
            self.imageDisplay.resize(self.imageDisplay.originalPixmap.size())


In [ ]:
# Cell 4: Launch the application in full-screen mode.
if __name__ == "__main__":
    application = QApplication(sys.argv)
    viewer = TiffViewer("testGeoTiff.tif")
    
    # Launch the viewer in full-screen mode.
    viewer.showMaximized()
    sys.exit(application.exec_())
